In [2]:
import pandas as pd
import numpy as np

In [3]:
!pip install -q pyspark==3.4.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [4]:
# Setting Spark Session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('FirstSession').config('spark.executor.memory', '2g').getOrCreate()
spark

In [5]:
# Import and print sample data with Pandas
df_pandas = pd.read_csv("/content/sample_data/california_housing_train.csv")
print(df_pandas.head())
print(df_pandas.shape)

   longitude  latitude  housing_median_age  total_rooms  total_bedrooms  \
0    -114.31     34.19                15.0       5612.0          1283.0   
1    -114.47     34.40                19.0       7650.0          1901.0   
2    -114.56     33.69                17.0        720.0           174.0   
3    -114.57     33.64                14.0       1501.0           337.0   
4    -114.57     33.57                20.0       1454.0           326.0   

   population  households  median_income  median_house_value  
0      1015.0       472.0         1.4936             66900.0  
1      1129.0       463.0         1.8200             80100.0  
2       333.0       117.0         1.6509             85700.0  
3       515.0       226.0         3.1917             73400.0  
4       624.0       262.0         1.9250             65500.0  
(17000, 9)


In [6]:
# Create Spark DF from Pandas DF
df_pyspark = spark.createDataFrame(df_pandas)
df_pyspark

DataFrame[longitude: double, latitude: double, housing_median_age: double, total_rooms: double, total_bedrooms: double, population: double, households: double, median_income: double, median_house_value: double]

In [7]:
# Print Schema
df_pyspark.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)



In [8]:
# Show top rows
df_pyspark.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|  -114.31|   34.19|              15.0|     5612.0|        1283.0|    1015.0|     472.0|       1.4936|           66900.0|
|  -114.47|    34.4|              19.0|     7650.0|        1901.0|    1129.0|     463.0|         1.82|           80100.0|
|  -114.56|   33.69|              17.0|      720.0|         174.0|     333.0|     117.0|       1.6509|           85700.0|
|  -114.57|   33.64|              14.0|     1501.0|         337.0|     515.0|     226.0|       3.1917|           73400.0|
|  -114.57|   33.57|              20.0|     1454.0|         326.0|     624.0|     262.0|        1.925|           65500.0|
+---------+--------+----

In [9]:
# Print last records, as it seems longitude is ordered ascending
df_pyspark.tail(5)

[Row(longitude=-124.26, latitude=40.58, housing_median_age=52.0, total_rooms=2217.0, total_bedrooms=394.0, population=907.0, households=369.0, median_income=2.3571, median_house_value=111400.0),
 Row(longitude=-124.27, latitude=40.69, housing_median_age=36.0, total_rooms=2349.0, total_bedrooms=528.0, population=1194.0, households=465.0, median_income=2.5179, median_house_value=79000.0),
 Row(longitude=-124.3, latitude=41.84, housing_median_age=17.0, total_rooms=2677.0, total_bedrooms=531.0, population=1244.0, households=456.0, median_income=3.0313, median_house_value=103600.0),
 Row(longitude=-124.3, latitude=41.8, housing_median_age=19.0, total_rooms=2672.0, total_bedrooms=552.0, population=1298.0, households=478.0, median_income=1.9797, median_house_value=85800.0),
 Row(longitude=-124.35, latitude=40.54, housing_median_age=52.0, total_rooms=1820.0, total_bedrooms=300.0, population=806.0, households=270.0, median_income=3.0147, median_house_value=94600.0)]

In [10]:
# Generating Beedroms ratio to Rooms
df_pyspark.withColumn('Bedrooms %', df_pyspark['total_bedrooms']/df_pyspark['total_rooms']*100).show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|        Bedrooms %|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+------------------+
|  -114.31|   34.19|              15.0|     5612.0|        1283.0|    1015.0|     472.0|       1.4936|           66900.0|22.861724875267285|
|  -114.47|    34.4|              19.0|     7650.0|        1901.0|    1129.0|     463.0|         1.82|           80100.0| 24.84967320261438|
|  -114.56|   33.69|              17.0|      720.0|         174.0|     333.0|     117.0|       1.6509|           85700.0|24.166666666666668|
|  -114.57|   33.64|              14.0|     1501.0|         337.0|     515.0|     226.0|       3.1917|           73400.0| 22.45169886742172|
|  -114.57|  

In [11]:
# Descriptive statistics for relevant columns
df_pyspark.select('total_rooms','total_bedrooms','population','households').describe().show()

+-------+------------------+-----------------+------------------+-----------------+
|summary|       total_rooms|   total_bedrooms|        population|       households|
+-------+------------------+-----------------+------------------+-----------------+
|  count|             17000|            17000|             17000|            17000|
|   mean| 2643.664411764706|539.4108235294118|1429.5739411764705|501.2219411764706|
| stddev|2179.9470714527715|421.4994515798648|1147.8529591595263|384.5208408559008|
|    min|               2.0|              1.0|               3.0|              1.0|
|    max|           37937.0|           6445.0|           35682.0|           6082.0|
+-------+------------------+-----------------+------------------+-----------------+



In [13]:
# Count locations with population over 30000
df_pyspark.filter(df_pyspark.population > 30000).count()

1

In [14]:
# Retrieve five lowest populations of the set
df_pyspark.sort(df_pyspark.population.asc()).show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|  -118.44|   34.04|              16.0|       18.0|           6.0|       3.0|       4.0|        0.536|          350000.0|
|  -117.79|   35.21|               4.0|        2.0|           2.0|       6.0|       2.0|        2.375|          137500.0|
|  -122.06|   37.39|              26.0|       18.0|           4.0|       8.0|       4.0|         3.75|          375000.0|
|  -117.76|   35.22|               4.0|       18.0|           3.0|       8.0|       6.0|        1.625|          275000.0|
|  -120.85|   37.75|              26.0|       28.0|           4.0|       9.0|       5.0|        1.625|           85000.0|
+---------+--------+----

In [16]:
# Write as a .parquet file in a new directory
df_pyspark.write.parquet('california_housing.parquet')

In [17]:
# Read .parquet from new directory
spark.read.parquet('california_housing.parquet', header=True).show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|  -118.44|   34.27|              35.0|      777.0|         187.0|    1022.0|     186.0|          3.4|          139600.0|
|  -118.44|   34.27|              29.0|     1701.0|         419.0|    1616.0|     371.0|       3.3603|          142400.0|
|  -118.44|   34.26|              34.0|      325.0|          60.0|     433.0|      83.0|       5.5124|          174300.0|
|  -118.44|   34.26|              34.0|     1102.0|         212.0|     949.0|     212.0|       4.0792|          165100.0|
|  -118.44|   34.26|              28.0|     1077.0|         288.0|    1377.0|     293.0|       3.9167|          153900.0|
+---------+--------+----

In [18]:
# Check missing values on housing_median_age column
df_pyspark.where(df_pyspark.housing_median_age.isNull()).count()

0

In [19]:
# Create Temporary View and execute SQL statement
df_pyspark.createOrReplaceTempView("table1")
spark.sql("SELECT AVG(housing_median_age) FROM table1").show()

+-----------------------+
|avg(housing_median_age)|
+-----------------------+
|      28.58935294117647|
+-----------------------+



In [20]:
# Check Temporary View
view_exists = spark.catalog.tableExists("table1")
view_exists

True

In [21]:
# Execute more complex query combining temporary views
df_pyspark.createOrReplaceTempView("table2")
spark.sql("SELECT * FROM table2 WHERE housing_median_age > (SELECT AVG(housing_median_age) FROM table1)").show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|  -114.58|   33.63|              29.0|     1387.0|         236.0|     671.0|     239.0|       3.3438|           74000.0|
|  -114.59|   34.83|              41.0|      812.0|         168.0|     375.0|     158.0|       1.7083|           48500.0|
|  -114.59|   33.61|              34.0|     4789.0|        1175.0|    3134.0|    1056.0|       2.1782|           58400.0|
|   -114.6|   34.83|              46.0|     1497.0|         309.0|     787.0|     271.0|       2.1908|           48100.0|
|  -114.61|   34.84|              48.0|     1291.0|         248.0|     580.0|     211.0|       2.1571|           48600.0|
|  -114.61|   34.83|    

In [23]:
# Drop Temporary Views
spark.catalog.dropTempView("table1")
spark.catalog.dropTempView("table2")

True

In [24]:
spark.stop()